In [1]:
import os
from path import Path
import pickle
import numpy as np
import pandas as pd
import spacy
import seaborn as sns; sns.set()
from sklearn.manifold import TSNE


In [2]:
!python -m spacy download en_core_web_md

/Users/josemonagas/anaconda3/envs/PythonData/bin/python: No module named spacy


In [8]:
data = 'Resources/mbti_clean.csv'
df = pd.read_csv(data, index_col="type")
df.head()

,posts
type,
INFJ,The last thing INFJ friend posted his facebo...
INFJ,Hello ENFJ Sorry hear your distress Its only...
INFJ,Prozac wellbrutin least thirty minutes movin...
INFJ,Basically come with three items youve determi...
INFJ,All things moderation Sims indeed video ga...


In [10]:
INFJ = df.loc["INFJ"]
INFJ

,posts
type,
INFJ,The last thing INFJ friend posted his facebo...
INFJ,Hello ENFJ Sorry hear your distress Its only...
INFJ,Prozac wellbrutin least thirty minutes movin...
INFJ,Basically come with three items youve determi...
INFJ,All things moderation Sims indeed video ga...
...,...
INFJ,What this thread has turned into more con...
INFJ,that what they call these days tongue And ...
INFJ,dont know whether not youre including the ...


In [4]:
embedding_dim = 50
X = np.empty((df.shape[0], embedding_dim))

In [5]:
def doc2vec(document: str):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(document)
    return doc.vector

In [11]:
nlp = spacy.load("en_core_web_md")

In [12]:
nlp.

spacy.lang.en.English

In [ ]:
# for i, doc in enumerate(df["posts"]):
#     if i % 1000 == 0:
#         print(f"Completed {i} conversions")
#     X[i] = doc2vec(doc)

In [ ]:
import pickle

with open("embeddings.pkl", "wb") as f:
    pickle.dump(X, f)

In [ ]:
if os.path.exists("embeddings.pkl"):
    with open("embeddings.pkl", "rb") as f:
        X = pickle.load(f)
else:
    for i, doc in enumerate(df["posts"]):
        if i % 1000 == 0:
            print(f"Completed {i} conversions")
        X[i] = doc2vec(doc)

In [ ]:
y = df["type"]

In [ ]:
X = X[0 : 1000] 
y = y[0 : 1000]

In [ ]:
# Split the dataset into training and testing sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear', class_weight='balanced', probability=True )

In [ ]:
# Train the model
model.fit(X_train, y_train)

In [ ]:
# Create predictions with the model
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

In [ ]:
# Assess the accuracy_score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
# Generate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


In [ ]:
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=df.type,
            yticklabels=df.type)
plt.xlabel('true label')
plt.ylabel('predicted label')